In [17]:
import geopandas as gpd
import pandas as pd
import numpy as np

Populating the interactive namespace from numpy and matplotlib


# Geographic data

In [20]:
# read data
epci = pd.read_csv(              # Établissement public de coopération intercommunale 
    'raw/geo/epcicom2020.csv.gz',
    compression='gzip',
    delimiter=',',
    quotechar='"',
    escapechar='\\'
)
geo = gpd.read_file('raw/geo/code-postal-code-insee-2015/code-postal-code-insee-2015.shp')  # insee / CP + commune boundaries

In [71]:
epci.head()

,dept,siren,raison_sociale,nature_juridique,mode_financ,nb_membres,total_pop_tot,total_pop_mun,dep_com,insee,siren_membre,nom_membre,ptot_2020,pmun_2020
0,01,200029999,CC Rives de l'Ain - Pays du Cerdon,CC,FPU,14,14887,14590,01,01304,210103040,Pont-d'Ain,2993,2964
1,01,200029999,CC Rives de l'Ain - Pays du Cerdon,CC,FPU,14,14887,14590,01,01199,210101994,Jujurieux,2200,2156
2,01,200029999,CC Rives de l'Ain - Pays du Cerdon,CC,FPU,14,14887,14590,01,01273,210102737,Neuville-sur-Ain,1784,1756
3,01,200029999,CC Rives de l'Ain - Pays du Cerdon,CC,FPU,14,14887,14590,01,01314,210103149,Priay,1763,1729
4,01,200029999,CC Rives de l'Ain - Pays du Cerdon,CC,FPU,14,14887,14590,01,01363,210103636,Saint-Jean-le-Vieux,1802,1723


In [72]:
geo.head()

,id_geofla,code_com,insee_com,nom_com,statut,x_chf_lieu,y_chf_lieu,x_centroid,y_centroid,z_moyen,...,total_pop_tot,total_pop_mun,dep_com,insee,siren_membre,nom_membre,ptot_2020,pmun_2020,nom_com_sans_arr,EPCI
0,COMMUNE00000000000033723,284,35284,SAINT-JOUAN-DES-GUERETS,Commune simple,333544.0,6844687.0,333790.0,6844750.0,36.0,...,85143.0,82989.0,35,35284,213502842.0,Saint-Jouan-des-Guérets,2715.0,2646.0,SAINT-JOUAN-DES-GUERETS,CA du Pays de Saint Malo Agglomération
1,COMMUNE00000000000033839,380,01380,SAINT-NIZIER-LE-BOUCHOUX,Commune simple,865058.0,6597766.0,866859.0,6597777.0,207.0,...,136286.0,132380.0,01,01380,210103800.0,Saint-Nizier-le-Bouchoux,692.0,680.0,SAINT-NIZIER-LE-BOUCHOUX,CA du Bassin de Bourg-en-Bresse
2,COMMUNE00000000000033863,273,68273,RIESPACH,Commune simple,1022384.0,6724328.0,1022398.0,6724218.0,426.0,...,48638.0,47554.0,68,68273,216802736.0,Riespach,683.0,643.0,RIESPACH,CC Sundgau
3,COMMUNE00000000000033881,274,21274,FOISSY,Commune simple,818437.0,6670274.0,817608.0,6670993.0,428.0,...,7360.0,7198.0,21,21274,212102743.0,Foissy,168.0,165.0,FOISSY,CC du Pays Arnay Liernais
4,COMMUNE00000000000033942,418,39418,PICARREAU,Commune simple,909498.0,6631828.0,910664.0,6632209.0,561.0,...,23006.0,21937.0,39,39418,213904188.0,Picarreau,104.0,101.0,PICARREAU,"CC Arbois, Poligny, Salins, Coeur du Jura"


In [23]:
# switch to a nicer projection
geo = geo.to_crs(epsg=3857)
# merge with epci data
geo = geo.merge(epci, how='left', left_on='insee_com', right_on='insee')

In [63]:
# have a new column to merge Paris, Lyon and Marseille's arrondissements
split_arr = geo.nom_com.str.split("-", n=1, expand=True)[0]
geo['nom_com_sans_arr'] = geo['nom_com'].where(
    ~(split_arr.isin({"PARIS", "LYON", "MARSEILLE"}) & geo.nom_com.str.endswith('-ARRONDISSEMENT')),
    other=split_arr
)

# create a new EPCI column that takes the name of the EPCI if the commune is in one or the name of the commune if not
geo['EPCI'] = geo.raison_sociale.where(geo.raison_sociale.notna(), other=geo.nom_com_sans_arr)
# special case for Grand Paris EPCI, to have a finer detail on "petite couronne" it seems valuable to split it by departement
geo['EPCI'] = geo['EPCI'].where(~(geo.EPCI == 'Métropole du Grand Paris'), other=(geo['EPCI'] + '-' + geo['code_dept']))

In [105]:
# save to shapefile
geo[[
    'code_postal', 'insee_com', 'nom_com', 'nom_de_la_c', 'nom_com_sans_arr',         # communal level
    'EPCI', 'code_cant', 'code_arr', 'code_dept', 'nom_dept', 'code_reg', 'nom_reg',  # subdivisions
    'superficie', 'population',                                                       # statistics
    'geometry'                                                                        # geometry (communal boundaries)
]].to_file("processed/geo/commune_insee_cp_epci_2020.shp")

# INSEE Death logs data

In [128]:
read data
death_history_raw = pd.read_csv(
    'raw/death/deces-1970-to-2020-03.csv.gz',
    compression='gzip',
    delimiter=';',
    quotechar='"',
    escapechar='\\',
    error_bad_lines=False,
    dtype={
        'datenaiss': 'str',
        'datedeces': 'str',
        'lieunaiss': 'str',
        'lieudeces': 'str',
    },
)
death_recent_raw = pd.read_csv(
    'raw/death/DC_jan2018-avr2020_det.csv.gz',
    compression='gzip',
    delimiter=';',
    quotechar='"',
    escapechar='\\',
    dtype={
        'ADEC': 'str',
        'MDEC': 'str',
        'JDEC': 'str',
        'ANAIS': 'str',
        'MNAIS': 'str',
        'JNAIS': 'str',
        'COMDEC': 'str',
    },
)


In [110]:
death_history_raw.head()

,nomprenom,sexe,datenaiss,lieunaiss,commnaiss,paysnaiss,datedeces,lieudeces,actedeces
0,DUCRET*MARIE ANTOINETTE/,2,19220109,1004,AMBERIEU-EN-BUGEY,NaN,19701210.0,1421,6
1,GRANGEON*ERIC JEAN REMY/,1,19690329,1004,AMBERIEU-EN-BUGEY,NaN,19700425.0,69383,1059
2,VELLET*PHILIPPE/,1,19700201,1004,AMBERIEU-EN-BUGEY,NaN,19700203.0,1004,12
3,PRESSAVIN*LYDIE/,2,19700406,1004,AMBERIEU-EN-BUGEY,NaN,19700406.0,1004,33
4,DOUAT*MARIE-SYLVIA MARTINE/,2,19700708,1004,AMBERIEU-EN-BUGEY,NaN,19700708.0,1053,457
...,...,...,...,...,...,...,...,...,...
25088224,DUSS*MIREILLE SYLVIE/,2,19570125,69383,LYON 3E ARRONDISSEMENT,NaN,20200301.0,99405,332
25088225,MARIANI*ALESSANDRO ENRICO/,1,19630105,99132,LONDRES,ROYAUME-UNI,20191110.0,99135,151
25088226,FAVRE*LOUIS GERMAIN/,1,19350509,54528,TOUL,NaN,20200219.0,99219,259
25088227,LE GUENNEC*GUY EUGENE MARIE/,1,19270322,56177,PLUVIGNER,NaN,20191126.0,99243,16


In [130]:
death_recent_raw.head()

,ADEC,MDEC,JDEC,DEPDEC,COMDEC,ANAIS,MNAIS,JNAIS,SEXE,DEPDOM,LIEUDEC2
0,2020,01,01,01,01014,1977,09,21,M,39,Autres
1,2020,01,01,01,01053,1922,09,28,F,01,HosMar
2,2020,01,01,01,01102,1995,03,11,M,01,Logem
3,2020,01,01,01,01179,1937,04,09,F,01,Logem
4,2020,01,01,01,01283,1930,08,05,F,01,HopCli


In [126]:
# format date of birth and death, fallback of unknown values for day / month to 1st of month / Jan 1st

death_history_raw['date_birth'] = pd.to_datetime(
    {
        'year': death_history_raw.datenaiss.str[:4],
        'month': death_history_raw.datenaiss.str[4:6].replace('00', '01'),
        'day': death_history_raw.datenaiss.str[6:].replace('00', '01')
    },
    errors='coerce'
)

death_history_raw['date_death'] = pd.to_datetime(
    {
        'year': death_history_raw.datedeces.str[:4],
        'month': death_history_raw.datedeces.str[4:6].replace('00', '01'),
        'day': death_history_raw.datedeces.str[6:].replace('00', '01')
    },
    errors='coerce'
)


In [132]:
# format date of birth and death, fallback of unknown values for day / month to 1st of month / Jan 1st

death_recent_raw['date_birth'] = pd.to_datetime(
    {
        'year': death_recent_raw.ANAIS,
        'month': death_recent_raw.MNAIS.str.replace('00', '01'),
        'day': death_recent_raw.JNAIS.str.replace('00', '01')
    },
    errors='coerce'
)

death_recent_raw['date_death'] = pd.to_datetime(
    {
        'year': death_recent_raw.ADEC,
        'month': death_recent_raw.MDEC,
        'day': death_recent_raw.JDEC
    },
    errors='raise'
)


In [195]:
# normalize name and gender content, compute age

split_nomprenom = death_history_raw.nomprenom.str.split('*', expand=True)
death_history_raw['lastname'] = split_nomprenom[0]
death_history_raw['firstname'] = split_nomprenom[1].str.strip('/')
death_history_raw['gender'] = death_history_raw['sexe'].map({1: 'M', 2: 'F'})
death_history_raw['age'] = ((death_history_raw.date_death - death_history_raw.date_birth) / pd.Timedelta(365.25, 'D')).fillna(-1).astype('int')

In [209]:
# normalize column names and save

death_history = death_history_raw.rename(columns={'lieunaiss': 'insee_code_birth', 'commnaiss': 'commune_birth', 'paysnaiss': 'country_birth', 'lieudeces': 'insee_code_death', 'actedeces': 'death_certificate_num'})[[
    'firstname', 'lastname', 'gender', 'age',
    'date_death', 'insee_code_death', 'death_certificate_num',
    'date_birth', 'insee_code_birth', 'commune_birth', 'country_birth',
]]
death_history.to_parquet('processed/death/insee_individual_history_1970_to_20200330.parquet')

In [229]:
# compute age

death_recent_raw['age'] = ((death_recent_raw.date_death - death_recent_raw.date_birth) / pd.Timedelta(365.25, 'D')).fillna(-1).astype('int')

In [235]:
# normalize column names and save

death_recent = death_recent_raw.rename(columns={'DEPDOM': 'home_dept_death', 'COMDEC': 'insee_code_death', 'SEXE': 'gender'})[[
    'gender', 'age',
    'date_death', 'insee_code_death', 'home_dept_death',
    'date_birth',
]]
death_recent.to_parquet('processed/death/insee_individual_recent_2018_to_20200427.parquet')

## Historical and weekly logs combination

In [217]:
columns = ['date_death', 'insee_code_death', 'age', 'gender']

In [236]:
death_log = pd.concat((
    death_history[columns][death_history.date_death < '2018-01-01'],
    death_recent[columns],
))
# limit to 1 line per date/insee code/age/gender then save
death_log_compress = death_log.groupby(columns, as_index=True).agg(nb_death=('_', 'size')).reset_index()
death_log_compress.to_parquet('processed/death/insee_date-place-age-gender_1970_to_20200427.parquet')

# Official Covid-19 related death counts

In [271]:
death_official_raw = pd.read_csv(
    'raw/death/chiffres-cles.csv.gz',
    compression='gzip',
    delimiter=',',
    quotechar='"',
    escapechar='\\'
)
death_official_raw.head()

,date,granularite,maille_code,maille_nom,cas_confirmes,cas_ehpad,cas_confirmes_ehpad,cas_possibles_ehpad,deces,deces_ehpad,reanimation,hospitalises,gueris,depistes,source_nom,source_url,source_archive,source_type
0,2020-01-24,departement,DEP-16,Charente,0.0,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,ARS Nouvelle-Aquitaine,https://www.nouvelle-aquitaine.ars.sante.fr/co...,NaN,agences-regionales-sante
1,2020-01-24,departement,DEP-17,Charente-Maritime,0.0,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,ARS Nouvelle-Aquitaine,https://www.nouvelle-aquitaine.ars.sante.fr/co...,NaN,agences-regionales-sante
2,2020-01-24,departement,DEP-19,Corrèze,0.0,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,ARS Nouvelle-Aquitaine,https://www.nouvelle-aquitaine.ars.sante.fr/co...,NaN,agences-regionales-sante
3,2020-01-24,departement,DEP-23,Creuse,0.0,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,ARS Nouvelle-Aquitaine,https://www.nouvelle-aquitaine.ars.sante.fr/co...,NaN,agences-regionales-sante
4,2020-01-24,departement,DEP-24,Dordogne,0.0,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,ARS Nouvelle-Aquitaine,https://www.nouvelle-aquitaine.ars.sante.fr/co...,NaN,agences-regionales-sante


In [272]:
death_official_dept = death_official_raw.loc[death_official_raw.granularite == 'departement']
death_official_domtom = death_official_raw.loc[death_official_raw.granularite == 'collectivite-outremer']

death_official_reg = death_official_raw.loc[death_official_raw.granularite == 'region']
death_official_national = death_official_raw.loc[death_official_raw.granularite == 'pays']

In [282]:
death_official_dept['code_dept'] = death_official_dept.maille_code.str.split('-', expand=True)[1]
death_official_domtom['code_dept'] = death_official_domtom.maille_code.str.split('-', expand=True)[1]
death_official_reg['code_reg'] = death_official_reg.maille_code.str.split('-', expand=True)[1]

<ipython-input-282-bf49d1696411>:1: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  death_official_dept['code_dept'] = death_official_dept.maille_code.str.split('-', expand=True)[1]
<ipython-input-282-bf49d1696411>:2: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  death_official_domtom['code_dept'] = death_official_domtom.maille_code.str.split('-', expand=True)[1]
<ipython-input-282-bf49d1696411>:3: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_i

In [283]:
death_official_reg

,date,granularite,maille_code,maille_nom,cas_confirmes,cas_ehpad,cas_confirmes_ehpad,cas_possibles_ehpad,deces,deces_ehpad,reanimation,hospitalises,gueris,depistes,source_nom,source_url,source_archive,source_type,code_reg
15,2020-01-24,region,REG-11,Île-de-France,2.0,NaN,NaN,NaN,NaN,NaN,NaN,2.0,NaN,NaN,Santé publique France,https://www.santepubliquefrance.fr/maladies-et...,https://web.archive.org/web/20200124202351/htt...,sante-publique-france,11
16,2020-01-24,region,REG-11,Île-de-France,2.0,NaN,NaN,NaN,NaN,NaN,NaN,2.0,NaN,NaN,Ministère des Solidarités et de la Santé,https://solidarites-sante.gouv.fr/IMG/pdf/2001...,https://web.archive.org/web/20200322183046/htt...,ministere-sante,11
17,2020-01-24,region,REG-75,Nouvelle-Aquitaine,1.0,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,ARS Nouvelle-Aquitaine,https://www.nouvelle-aquitaine.ars.sante.fr/co...,NaN,agences-regionales-sante,75
18,2020-01-24,region,REG-75,Nouvelle-Aquitaine,1.0,NaN,NaN,NaN,NaN,NaN,NaN,1.0,NaN,NaN,Santé publique France,https://www.santepubliquefrance.fr/maladies-et...,https://web.archive.org/web/20200124202351/htt...,sante-publique-france,75
19,2020-01-24,region,REG-75,Nouvelle-Aquitaine,1.0,NaN,NaN,NaN,NaN,NaN,NaN,1.0,NaN,NaN,Ministère des Solidarités et de la Santé,https://solidarites-sante.gouv.fr/IMG/pdf/2001...,https://web.archive.org/web/20200322183046/htt...,ministere-sante,75
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
9315,2020-05-07,region,REG-75,Nouvelle-Aquitaine,NaN,NaN,NaN,NaN,347.0,NaN,85.0,521.0,1589.0,NaN,OpenCOVID19-fr,NaN,NaN,opencovid19-fr,75
9316,2020-05-07,region,REG-76,Occitanie,NaN,NaN,NaN,NaN,448.0,NaN,103.0,466.0,2290.0,NaN,OpenCOVID19-fr,NaN,NaN,opencovid19-fr,76
9317,2020-05-07,region,REG-84,Auvergne-Rhône-Alpes,NaN,NaN,NaN,NaN,1462.0,NaN,276.0,2206.0,5596.0,NaN,OpenCOVID19-fr,NaN,NaN,opencovid19-fr,84
9318,2020-05-07,region,REG-93,Provence-Alpes-Côte d'Azur,NaN,NaN,NaN,NaN,764.0,NaN,184.0,1337.0,4187.0,NaN,OpenCOVID19-fr,NaN,NaN,opencovid19-fr,93


In [276]:
death_official_domtom.code_dept.unique()

array(['977', '978', '987', '974', '988', '986'], dtype=object)

In [278]:
death_official_reg['code_reg'] = death_official_reg['']

,date,granularite,maille_code,maille_nom,cas_confirmes,cas_ehpad,cas_confirmes_ehpad,cas_possibles_ehpad,deces,deces_ehpad,reanimation,hospitalises,gueris,depistes,source_nom,source_url,source_archive,source_type
15,2020-01-24,region,REG-11,Île-de-France,2.0,NaN,NaN,NaN,NaN,NaN,NaN,2.0,NaN,NaN,Santé publique France,https://www.santepubliquefrance.fr/maladies-et...,https://web.archive.org/web/20200124202351/htt...,sante-publique-france
16,2020-01-24,region,REG-11,Île-de-France,2.0,NaN,NaN,NaN,NaN,NaN,NaN,2.0,NaN,NaN,Ministère des Solidarités et de la Santé,https://solidarites-sante.gouv.fr/IMG/pdf/2001...,https://web.archive.org/web/20200322183046/htt...,ministere-sante
17,2020-01-24,region,REG-75,Nouvelle-Aquitaine,1.0,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,ARS Nouvelle-Aquitaine,https://www.nouvelle-aquitaine.ars.sante.fr/co...,NaN,agences-regionales-sante
18,2020-01-24,region,REG-75,Nouvelle-Aquitaine,1.0,NaN,NaN,NaN,NaN,NaN,NaN,1.0,NaN,NaN,Santé publique France,https://www.santepubliquefrance.fr/maladies-et...,https://web.archive.org/web/20200124202351/htt...,sante-publique-france
19,2020-01-24,region,REG-75,Nouvelle-Aquitaine,1.0,NaN,NaN,NaN,NaN,NaN,NaN,1.0,NaN,NaN,Ministère des Solidarités et de la Santé,https://solidarites-sante.gouv.fr/IMG/pdf/2001...,https://web.archive.org/web/20200322183046/htt...,ministere-sante
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
9315,2020-05-07,region,REG-75,Nouvelle-Aquitaine,NaN,NaN,NaN,NaN,347.0,NaN,85.0,521.0,1589.0,NaN,OpenCOVID19-fr,NaN,NaN,opencovid19-fr
9316,2020-05-07,region,REG-76,Occitanie,NaN,NaN,NaN,NaN,448.0,NaN,103.0,466.0,2290.0,NaN,OpenCOVID19-fr,NaN,NaN,opencovid19-fr
9317,2020-05-07,region,REG-84,Auvergne-Rhône-Alpes,NaN,NaN,NaN,NaN,1462.0,NaN,276.0,2206.0,5596.0,NaN,OpenCOVID19-fr,NaN,NaN,opencovid19-fr
9318,2020-05-07,region,REG-93,Provence-Alpes-Côte d'Azur,NaN,NaN,NaN,NaN,764.0,NaN,184.0,1337.0,4187.0,NaN,OpenCOVID19-fr,NaN,NaN,opencovid19-fr
